# AZ Identity Secret Management

## Prerequisite tasks

To set up and run this example, you must first set up the following:

1. Install AZ CLI as described in the **"3.0.AZ_CLI_Installation"** notebook.
2. Install azure-identity and azure-keyvault-secrets libraries in your environment as described in the **"3.1.AZ_python_library_Installation"** notebook (if it is not already done)
3. Login to the AZ account to operate using the **"3.9.Connect to the AZ target system"** notebook

In [ ]:
%run "3.9. Connect to the AZ target system.ipynb"

### Create a resource group and a key vault for the example

In [ ]:
!az group create --name myResourceGroup --location "EastUS"

In [ ]:
!az keyvault create --name aTestKeyvault --resource-group myResourceGroup --location "EastUS"

### Set Environmental variable KEY_VAULT_NAME

In [ ]:
!export KEY_VAULT_NAME=aTestKeyvault

In [ ]:
%env KEY_VAULT_NAME =aTestKeyvault

### Grant access to the key vault 

(modify as required)

In [ ]:
!az keyvault set-policy --name aTestKeyvault --upn user@domain --secret-permissions delete get list set

### Basic operations over the secret value

The following example shows how to:
Retrieve a secret value using [get_secret_value](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/secretsmanager.html#SecretsManager.Client.get_secret_value) funtion

In [ ]:
import os
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential

In [ ]:
def put_secret():

    keyVaultName = os.environ["KEY_VAULT_NAME"]
    KVUri = f"https://{keyVaultName}.vault.azure.net"

    credential = DefaultAzureCredential()
    client = SecretClient(vault_url=KVUri, credential=credential)

    secretName = input("Input a name for your secret > ")
    secretValue = input("Input a value for your secret > ")

    print(f"Creating a secret in {keyVaultName} called '{secretName}' with the value '{secretValue}' ...")
    client.set_secret(secretName, secretValue)
    print(" done.")

In [ ]:
def get_secret():

    keyVaultName = os.environ["KEY_VAULT_NAME"]
    KVUri = f"https://{keyVaultName}.vault.azure.net"

    credential = DefaultAzureCredential()
    client = SecretClient(vault_url=KVUri, credential=credential)

    secretName = input("Input the name of your secret > ")
    
    print(f"Retrieving your secret {secretName} from {keyVaultName}.")
    retrieved_secret = client.get_secret(secretName)
    print(f"Your secret is '{retrieved_secret.value}'.")
    print(" done.")
    
    poller = client.begin_delete_secret(secretName)
    deleted_secret = poller.result()

    print(" done.")

In [ ]:
def delete_secret():

    keyVaultName = os.environ["KEY_VAULT_NAME"]
    KVUri = f"https://{keyVaultName}.vault.azure.net"

    credential = DefaultAzureCredential()
    client = SecretClient(vault_url=KVUri, credential=credential)

    secretName = input("Input the name of your secret > ")

    print(f"Deleting your secret {secretName} from {keyVaultName} ...")
    poller = client.begin_delete_secret(secretName)
    deleted_secret = poller.result()

    print(" done.")

In [ ]:
put_secret()

In [ ]:
get_secret()

In [ ]:
delete_secret()

**Note**:That it is the same as executing with the CLI

In [ ]:
!az keyvault secret set --vault-name aTestKeyvault --name "aSampleTestSecret" --value "3$5&7(9=" 

In [ ]:
!az keyvault secret show --vault-name aTestKeyvault --name "aSampleTestSecret" --query "value" 

In [ ]:
!az keyvault secret delete --vault-name aTestKeyvault --name "aSampleTestSecret" 

### Clean up resources allocated

To avoid any additional extra cost, it is recommended to clean the resources after finish the Lab.

In [ ]:
!az group delete --resource-group myResourceGroup